## A Game Score Standard Deviation Model of NCCA Men's Team Scoring.

In this notebook I fit a Game Score Standard Deviation model to the home team margin of victory.
I use the method described in Statistical Sports Models in Excel (Mack, 2019), ported from Mack's Excel model to Python by me.

In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
import warnings
from scipy.optimize import least_squares

In [2]:
# define functions for calculating model error metrics
def sum_squared_error(predictions, targets):
    return sum((predictions - targets) * (predictions - targets))

# coefficient of determination, r-squared; shows percentage variation in y which is explained by all the x variables together
def r_squared(predictions, targets):
    y_mean_line = [np.mean(targets) for y in targets]
    squared_error_regression = sum_squared_error(predictions, targets)
    squared_error_y_mean = sum_squared_error(targets, y_mean_line)
    return 1 - (squared_error_regression / squared_error_y_mean)

# root mean square error
def rmse(predictions, targets):
    return np.sqrt(((predictions-targets)**2).mean())

# mean absolute error
def mae(predictions, targets):
    return abs(predictions - targets).mean()

def brier_score(projected_probabilities, actual_results):
    prob_error_sq = (actual_results - projected_probabilities)**2
    return sum(prob_error_sq) / len(prob_error_sq)

def log_loss(projected_probabilities, actual_results):
    log_error = actual_results*np.log(projected_probabilities) + (1-actual_results)*np.log(1-projected_probabilities)
    return sum(log_error) * -1 / len(log_error)

In [3]:
# create the Game Score Standard Deviation model framework:

class gssd_mov:
    """
        Fits a Game Score Standard Deviation model to the home team margin of victory.
        Uses the method described in Statistical Sports Models in Excel (Mack, 2019).
        Inputs are a 2 separate lists of strings for away and home team names,
        and 2 separate lists of integers for away and home team scores.
        
        Briefly, 4 statistics are calculated for each team:
        (average points for home, average points against home, 
        average points for away, average points against away)
        Then an Ordinary Least Squares minimization is used to calculate the coefficients.
        The points stats are stored in .points_df, and the optimized coefficients stored as
        .opt_intercept, .opt_pfh, .opt_pah, .opt_pfa, .opt_paa.
        
        Based on those stats and coefficients, the model predicts a home margin of victory with .predict_raw()
        The next steps described in SSME are 
        i: run a linear regression to map the model predictions to actual mov, .predict_mov()
        ii: input the regression estimated spread into the cumulative density function of the 
            normal distribution to get the estimated home team win percentage, .predict_proba_()
        Both these steps are performed inside this module, but it is recommended that you use the 
        .predict_raw() method to get the GSSD model's pure prediction, and then run your own linear
        regression on that to see a full regression summary output, as well as for use in an ensemble.
    """
    def fit(self, away_teams, home_teams, away_scores, home_scores):
        self.a_teams = sorted(list(set(away_teams))) # list of unique away team names, sorted alphabetically
        self.h_teams = sorted(list(set(home_teams))) # list of unique home team names, sorted alphabetically
        if self.a_teams == self.h_teams:
            teams = set(self.a_teams+self.h_teams)
            self.teams = sorted(list(teams))
        else:
            warnings.warn(f'''The list of away teams does not match the list of home teams:
                          {len(self.a_teams)} unique away team names found.
                          {len(self.h_teams)} unique home team names found.
                          This can happen when one or more teams you provided only appears in the home or away column,
                          and can cause an NaN strength rating to appear in the opposite column.''')
            teams = set(self.a_teams+self.h_teams)
            self.teams = sorted(list(teams))
        # create a dataframe of the average points scored for and against each team at home and away
        temp_array = np.zeros((len(self.teams),4))
        temp_df = pd.DataFrame({'home team':home_teams,'away team':away_teams,
                                'home final':home_scores,'away final':away_scores})
        for index,team in enumerate(self.teams):
            # average points for, when home
            pfh = round(temp_df[temp_df['home team']==team]['home final'].mean(), 4)
            # average points against, when home
            pah = round(temp_df[temp_df['home team']==team]['away final'].mean(), 4)
            # average points for, when away
            pfa = round(temp_df[temp_df['away team']==team]['away final'].mean(), 4)
            # average points against, when away
            paa = round(temp_df[temp_df['away team']==team]['home final'].mean(), 4)
            temp_array[index] = [pfh,pah,pfa,paa]
        points_df = pd.DataFrame(temp_array,columns=['pfh','pah','pfa','paa'])
        points_df['team'] = self.teams
        self.points_df = points_df[['team','pfh','pah','pfa','paa']]
        del temp_array
        # create a dataframe for the LS optimization, using the away team's (pfa, paa) and home team's (pfh, pah)
        opt_df = temp_df.merge(points_df[['team','pfa','paa']], how='left',left_on='away team',right_on='team')
        opt_df = opt_df.merge(points_df[['team','pfh','pah']], how='left',left_on='home team',right_on='team')
        opt_df['home mov actual'] = opt_df['home final'] - opt_df['away final']
        # optimize pfh, pah, pfa, and paa coefficients to the home team's margin of victory
        pred_mov = lambda x, a, b, c, d: x[0] + x[1]*a + x[2]*b + x[3]*c + x[4]*d
        def error(x, a, b, c, d, actual_mov):
            return (pred_mov(x,a,b,c,d) - actual_mov)
        # the unkown vector of parameters:
        # x[0] intercept, x[1] pfh coefficient, x[2] pah coefficient, x[3] pfa coefficient, x[4] paa coefficient
        a,b = opt_df['pfh'].values, opt_df['pah'].values
        c,d = opt_df['pfa'].values,opt_df['paa'].values
        y = opt_df['home mov actual'].values
        x0 = np.array([1,1,1,1,1]) #dummy weights
        res = least_squares(error, x0, args=(a,b,c,d,y), verbose=0)
        # store the optimized coefficients:
        self.opt_intercept, self.opt_pfh, self.opt_pah, self.opt_pfa, self.opt_paa = res.x
        del temp_df
        # next, perform linear regression to map the points based predictions to actual mov
        opt_df['pred mov'] = self.opt_intercept+(self.opt_pfh*opt_df['pfh'])+(self.opt_pah*opt_df['pah'])+(self.opt_pfa*opt_df['pfa'])+(self.opt_paa*opt_df['paa'])
        self.pred_mov = opt_df['pred mov'].values
        X = opt_df['pred mov']
        X = sm.add_constant(X)
        y = opt_df['home mov actual']
        ols_reg = sm.OLS(y, X)
        lin_reg = ols_reg.fit()
        self.model_r2 = lin_reg.rsquared
        self.lr_intercept, self.lr_coefficient = lin_reg.params
        self.model_error = lin_reg.resid.std()

    def predict_raw(self, away_team, home_team):
        """
        Raw GSSD model predicted home margin of victory.
        Note that this prediction is ACTUAL mov.
        Thus a prediction of 3.0 means the home team is favored to win by 3, the equivalent betting market spread being -3.
        """
        if type(away_team) == str and type(home_team) == str:
            pfh = self.points_df[self.points_df['team']==home_team]['pfh'].values[0]
            pah = self.points_df[self.points_df['team']==home_team]['pah'].values[0]
            pfa = self.points_df[self.points_df['team']==away_team]['pfa'].values[0]
            paa = self.points_df[self.points_df['team']==away_team]['paa'].values[0]
            pred_mov = self.opt_intercept+(self.opt_pfh*pfh)+(self.opt_pah*pah)+(self.opt_pfa*pfa)+(self.opt_paa*paa)
        else:
            matchups = list(zip(away_team, home_team))
            predictions = []
            for match in matchups:
                pfh = self.points_df[self.points_df['team']==match[1]]['pfh'].values[0]
                pah = self.points_df[self.points_df['team']==match[1]]['pah'].values[0]
                pfa = self.points_df[self.points_df['team']==match[0]]['pfa'].values[0]
                paa = self.points_df[self.points_df['team']==match[0]]['paa'].values[0]
                predictions.append(self.opt_intercept+(self.opt_pfh*pfh)+(self.opt_pah*pah)+(self.opt_pfa*pfa)+(self.opt_paa*paa))
            pred_mov = predictions 

        return np.array(pred_mov)

    def predict_mov(self, away_team, home_team):
        """
        Regression estimated spread.
        Note that this prediction is for the ACTUAL home mov.
        Thus a prediction of 3.0 means the home team is favored to win by 3, the equivalent betting market spread being -3.
        """
        pred_mov = self.predict_raw(away_team, home_team)
        est_spread = self.lr_intercept+(self.lr_coefficient*pred_mov)
        return est_spread

    def predict_proba_(self, away_team, home_team, margin=0.0):
        """
        The probability of covering the spread.
        Note that this function takes as input the predicted margin, which uses the opposite sign of the spread.
        For example, if the bookmaker's offered spread is -3, you flip the sign and input +3 here.
        """
        est_spread = self.predict_mov(away_team, home_team)
        win_pct = 1-norm.cdf(margin, est_spread, self.model_error)
        return win_pct

## Load and Prep the Data

In [4]:
# the data used in this exercise was originaly scraped from the scores and odds archives available at:
# www.sportsbookreviewsonline.com
df = pd.read_csv('ncaam_scrubbed_2015to2020.csv')
df.head()

,date,away court,away team,away ml,away dec,away final,home court,home team,home ml,home dec,home final,market total,total actual,market mov,home mov actual,home win
0,20151113,V,Colorado,475,5.75,62,H,IowaState,-600,1.166667,68,150.5,130,10.5,6,1
1,20151113,V,Elon,175,2.75,85,H,CharlotteU,-205,1.487805,74,146.5,159,4.5,-11,0
2,20151113,V,Drexel,415,5.15,81,H,St.Josephs,-540,1.185185,82,127.5,163,10.0,1,1
3,20151113,V,JamesMadison,365,4.65,87,H,Richmond,-465,1.215054,75,137.5,162,9.5,-12,0
4,20151113,V,Dartmouth,535,6.35,67,H,SetonHall,-685,1.145985,84,136.0,151,11.0,17,1


In [5]:
# in this exercise I will ignore all games played on a neutral court
drop_idx = df[df['away court']=='N'].index
df.drop(drop_idx, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19888 entries, 0 to 19887
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             19888 non-null  int64  
 1   away court       19888 non-null  object 
 2   away team        19888 non-null  object 
 3   away ml          19888 non-null  int64  
 4   away dec         19888 non-null  float64
 5   away final       19888 non-null  int64  
 6   home court       19888 non-null  object 
 7   home team        19888 non-null  object 
 8   home ml          19888 non-null  int64  
 9   home dec         19888 non-null  float64
 10  home final       19888 non-null  int64  
 11  market total     19888 non-null  float64
 12  total actual     19888 non-null  int64  
 13  market mov       19888 non-null  float64
 14  home mov actual  19888 non-null  int64  
 15  home win         19888 non-null  int64  
dtypes: float64(4), int64(8), object(4)
memory usage: 2.4+ MB


In [7]:
# add some columns to use in checking the accuracy of the closing line
df['home no vig'] = (1/df['home dec']) / ((1/df['home dec'])+(1/df['away dec']))

cond_list = [df['home no vig'] > 0.5]
choice_list = [1]
df['market class'] = np.select(cond_list, choice_list)

cond_list = [df['home win']==df['market class']]
choice_list = [1]
df['mkt correct class'] = np.select(cond_list, choice_list)

## Train and Test the Model

In [8]:
# make a train test split of the team scoring data
train_len = int(len(df)*0.80) # use first 80% of entries to train the model
train_df = df.iloc[:train_len].copy()
test_df = df.iloc[train_len:].copy()

In [9]:
# instantiate the game score standard deviation class
gssd = gssd_mov()

# fit the model to training data
gssd.fit(train_df['away team'].values,train_df['home team'].values,train_df['away final'].values,train_df['home final'].values)

In [10]:
# check the head of the team scoring dataframe
# shows the average points scored for and against each team when away or home
gssd.points_df.head()

,team,pfh,pah,pfa,paa
0,AbileneChristian,77.3333,67.5000,66.5263,67.5263
1,AirForce,72.7885,72.9231,63.5294,74.9020
2,Akron,74.7963,66.1667,72.1273,75.8727
3,Alabama,71.9848,67.4091,67.1667,71.8750
4,AlabamaA&M,63.8462,66.3846,58.0645,79.7097


In [11]:
# make predictions on the test set:
test_df['pred mov'] = gssd.predict_raw(test_df['away team'].values, test_df['home team'].values)

In [12]:
# add columns needed to calculate model performance metrics
test_df['reg est spread'] = gssd.lr_intercept + (gssd.lr_coefficient * test_df['pred mov'])
test_df['win prob'] = 1-norm.cdf(0, test_df['reg est spread'], gssd.model_error)

cond_list = [test_df['win prob'] > 0.5]
choice_list = [1]
test_df['raw class'] = np.select(cond_list, choice_list)

cond_list = [test_df['home mov actual'] > 0]
choice_list = [1]
test_df['game result'] = np.select(cond_list, choice_list)

cond_list = [test_df['game result']==test_df['raw class']]
choice_list = [1]
test_df['correct class'] = np.select(cond_list, choice_list)

test_df['pred error'] = test_df['reg est spread']-test_df['home mov actual']

In [13]:
print(f"The GSSD model achieved a {test_df['correct class'].sum()/len(test_df)*100:.4}% out-of-sample raw classification accuracy.")

The GSSD model achieved a 66.59% out-of-sample raw classification accuracy.


In [14]:
print("Model Error Metrics:")
out_samp_r2 = r_squared(test_df['reg est spread'],test_df['home mov actual'])
print(f'out-of-sample r-squared = {out_samp_r2:0.4}')
out_samp_rmse = rmse(test_df['reg est spread'],test_df['home mov actual'])
print(f'out-of-sample rmse = {out_samp_rmse:0.4}')
out_samp_mae = mae(test_df['reg est spread'],test_df['home mov actual'])
print(f'out-of-sample mae = {out_samp_mae:0.4}')
out_samp_brier = brier_score(test_df['win prob'], test_df['game result'])
print(f'out-of-sample Brier Score = {out_samp_brier:0.4}')
out_samp_ll = log_loss(test_df['win prob'], test_df['game result'])
print(f'out-of-sample Log Loss = {out_samp_ll:0.4}')

Model Error Metrics:
out-of-sample r-squared = 0.1228
out-of-sample rmse = 12.56
out-of-sample mae = 9.971
out-of-sample Brier Score = 0.2129
out-of-sample Log Loss = 0.6186


In [15]:
#calculate market performance metrics
home_ml_logloss = log_loss(test_df['home no vig'], test_df['home win'])
home_ml_brier = brier_score(test_df['home no vig'], test_df['home win'])
ats_rmse = rmse(test_df['market mov'], df['home mov actual'])
ats_mae = mae(test_df['market mov'], df['home mov actual'])

print('During the same subset of games, the market closing lines performed as follows:')
print(f"The betting market achieved a {test_df['mkt correct class'].sum()/len(test_df)*100:.2f}% raw classification accuracy.")
print(f'The home moneyline had a log loss of {home_ml_logloss:.4f}, and a Brier Score of {home_ml_brier:.4f} .')
print(f'ATS rmse: {ats_rmse:.2f}, mae: {ats_mae:.2f}')

During the same subset of games, the market closing lines performed as follows:
The betting market achieved a 72.07% raw classification accuracy.
The home moneyline had a log loss of 0.5387, and a Brier Score of 0.1819 .
ATS rmse: 11.13, mae: 8.70


## Conclusion

Upon comparing the model's cliassification accuracy and error metrics from the training set to the market closing line for the same set of games, clearly the market closing line performs better than the model and this would not make a profitable betting model in this situation.